In [4]:
import pandas as pd

In [16]:
movie= pd.read_csv(r"C:\Users\Karth\Downloads\Machine learning\Datasets\Movies1\movies1.csv")
rating= pd.read_csv(r"C:\Users\Karth\Downloads\Machine learning\Datasets\Movies1\ratings.csv")

In [18]:
movie.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [7]:
rating.head(1)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703


In [11]:
movie.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [12]:
rating.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [19]:
data = pd.merge(rating, movie, on='movieId')
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [20]:
data[data['movieId']==2]

,userId,movieId,rating,timestamp,title,genres
560,6,2,4.0,845553522,Jumanji (1995),Adventure|Children|Fantasy
1026,8,2,4.0,839463806,Jumanji (1995),Adventure|Children|Fantasy
1773,18,2,3.0,1455617462,Jumanji (1995),Adventure|Children|Fantasy
2275,19,2,3.0,965704331,Jumanji (1995),Adventure|Children|Fantasy
2977,20,2,3.0,1054038313,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...,...
95102,600,2,4.0,1237764627,Jumanji (1995),Adventure|Children|Fantasy
95965,602,2,4.0,840875851,Jumanji (1995),Adventure|Children|Fantasy
97044,604,2,5.0,832080293,Jumanji (1995),Adventure|Children|Fantasy
97144,605,2,3.5,1277176522,Jumanji (1995),Adventure|Children|Fantasy


In [23]:
data.groupby('title')['rating'].mean().sort_values(ascending=False).head()

title
Gena the Crocodile (1969)              5.0
True Stories (1986)                    5.0
Cosmic Scrat-tastrophe (2015)          5.0
Love and Pigeons (1985)                5.0
Red Sorghum (Hong gao liang) (1987)    5.0
Name: rating, dtype: float64

In [24]:
data.groupby('title')['rating'].mean().sort_values(ascending=False).tail()

title
Don't Look Now (1973)                      0.5
Journey 2: The Mysterious Island (2012)    0.5
Joe Dirt 2: Beautiful Loser (2015)         0.5
Jesus Christ Vampire Hunter (2001)         0.5
Fullmetal Alchemist 2018 (2017)            0.5
Name: rating, dtype: float64

In [25]:
data.groupby('title')['userId'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: userId, dtype: int64

In [27]:
user_freq= data[['userId', 'movieId']].groupby('userId').count().reset_index().rename({'movieId': 'n_movies'}).head()
user_freq

,userId,movieId
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [28]:
movie.movieId.count()

np.int64(9742)

In [29]:
data['genres']

0         Adventure|Animation|Children|Comedy|Fantasy
1                                      Comedy|Romance
2                               Action|Crime|Thriller
3                                    Mystery|Thriller
4                              Crime|Mystery|Thriller
                             ...                     
100831                          Drama|Horror|Thriller
100832                          Action|Crime|Thriller
100833                                         Horror
100834                                  Action|Sci-Fi
100835                    Action|Crime|Drama|Thriller
Name: genres, Length: 100836, dtype: object

In [30]:
unique_genres= set()

for genres in data['genres']:
    genre_list= genres.split('|')
    for genre in genre_list:
        unique_genres.add(genre)

In [32]:
all_genres= sorted(list(unique_genres))
all_genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [33]:
def genre_to_vector(genres, all_genres):
    vector = [0] * len(all_genres)
    genre_list = genres.split('|')
    for genre in genre_list:
        if genre in all_genres:
            index = all_genres.index(genre)
        vector[index] = 1
    return vector

In [34]:

genre_to_vector(data['genres'][1], all_genres)

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [35]:
movieDict = {}
for index, row in data.iterrows():
    movieId = row['movieId']
    title = row['title']
    genre_vector = genre_to_vector(row['genres'], all_genres)
    rating = row['rating']
    movieDict[movieId] = (title, genre_vector, rating)

In [36]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 5.0)

In [37]:
from scipy.spatial.distance import cosine

In [38]:
def compute_distence(movie_a, movie_b):
    genre_distance = cosine(movie_a[1], movie_b[1])
    rating_distance = abs(movie_a[2] - movie_b[2])
    total_distance = genre_distance+rating_distance
    return total_distance, genre_distance, rating_distance

In [39]:
import operator
def get_neighbors(movie_id, k):
    distances = []
    for movie in movieDict:
        if movie != movie_id:
            dist = compute_distence(movieDict[movie_id], movieDict[movie])
            distances.append((movie, dist))

    distances.sort(key=operator.itemgetter(1))

    neighbors = []
    for i in range(min(k, len(distances))):
        neighbors.append(distances[i][0])
    return neighbors

In [41]:
movie_id = int(input("Enter the movie ID: "))

neighbors = get_neighbors(movie_id, k=5)
avg_rating = 0
for neighbor in neighbors:
    avg_rating += movieDict[neighbor][2]
    print(f"{movieDict[neighbor][0]} {movieDict[neighbor][2]}")

if neighbors:
    avg_rating /= len(neighbors)
    print(f"Average Rating of Nearest Neighbors: {avg_rating}")
else:
    print("No neighbors found.")

Wedding Singer, The (1998) 2.5
Pretty Woman (1990) 2.5
Mallrats (1995) 2.5
Holiday, The (2006) 2.5
How to Lose a Guy in 10 Days (2003) 2.5
Average Rating of Nearest Neighbors: 2.5
